<a href="https://colab.research.google.com/github/jatinkchaudhary/Speech-Processing-Final-Year-Project-/blob/main/LSTM_RAVDESS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Necessary Imports 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
import numpy as np
import argparse
import time
import tensorflow as tf
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.utils import to_categorical

Preprocessing of the Dataset 

In [ ]:
dataset = pd.read_csv('features_1.csv', low_memory=False)

print(dataset.shape)

for i in range(0, len(dataset['0.1'])):
  if dataset.loc[i, '0.1'] == 'neutral':
    dataset.loc[i, '0.1'] = 1
  
  if dataset.loc[i, '0.1'] == 'calm':
    dataset.loc[i, '0.1'] = 2
  
  if dataset.loc[i, '0.1'] == 'happy':
    dataset.loc[i, '0.1'] = 3
  
  if dataset.loc[i, '0.1'] == 'sad':
    dataset.loc[i, '0.1'] = 4
  
  if dataset.loc[i, '0.1'] == 'angry':
    dataset.loc[i, '0.1'] = 5
  
  if dataset.loc[i, '0.1'] == 'fear':
    dataset.loc[i, '0.1'] = 6
  
  if dataset.loc[i, '0.1'] == 'disgust':
    dataset.loc[i, '0.1'] = 7
   
  if dataset.loc[i, '0.1'] == 'surprise':
    dataset.loc[i, '0.1'] = 8
   
#dataset.replace(to_replace = float('Nan'), value = 0)   

Y = dataset[['0.1']]

mfcc = pd.read_csv('mfcc.csv', low_memory=False)
mfcc = mfcc.drop(['emotion', 'gender'], axis = 1)
for i in range (0, len(dataset['0.1'])):
  mfcc.loc[i, 'mel_spectrogram'] = mfcc.loc[i, 'mel_spectrogram'].replace('\n', '')
  mfcc.loc[i, 'mel_spectrogram'] = mfcc.loc[i, 'mel_spectrogram'].replace('[', '')
  mfcc.loc[i, 'mel_spectrogram'] = mfcc.loc[i, 'mel_spectrogram'].replace(']', '')
mfcc = np.asarray(mfcc).astype(np.float)
print(mfcc.shape)

#X = dataset.drop(['0.1'], axis=1)

#mfcc.mel_spectrogram.apply(type)

#print(mfcc.head)

#X = (X - np.min(X))/(np.max(X) - np.min(X)) + 0.0001


(1440, 4)


ValueError: ignored

Split the Data into testing and training parts

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = None, shuffle = True)

In [ ]:
X_train = np.asarray(X_train).astype('float32')
Y_train = np.asarray(Y_train).astype('float32')
X_test = np.asarray(X_test).astype('float32')
Y_test = np.asarray(Y_test).astype('float32')
look_back =1

In [ ]:
print(dataset)

              0          1          2  ...        257        258  0.1
0    -76.384773 -76.384773 -76.384773  ...   0.000000   0.000000    1
1    -75.268448 -75.268448 -75.268448  ...   0.000000   0.000000    1
2    -79.611420 -79.611420 -79.611420  ... -79.611420 -79.611420    2
3    -76.828041 -76.828041 -76.828041  ... -76.805473 -76.828041    2
4    -80.147377 -80.147377 -80.147377  ... -80.147377 -80.147377    2
...         ...        ...        ...  ...        ...        ...  ...
1435 -73.414032 -73.414032 -73.414032  ...   0.000000   0.000000    8
1436 -59.929165 -59.947273 -59.938259  ... -58.620701 -58.794323    7
1437 -66.861420 -66.861420 -64.904045  ...   0.000000   0.000000    8
1438 -68.237564 -67.372169 -66.390640  ... -58.785629 -59.667213    7
1439 -63.501076 -63.501076 -63.420914  ... -63.217319 -63.165489    8

[1440 rows x 260 columns]


LSTM model

In [ ]:
model = Sequential()

np.random.seed(0)

X_train = X_train.reshape(-1, 1, 259)
Y_train = Y_train.reshape(-1, 1, 1)
X_test = X_test.reshape(-1, 1, 259)
Y_test = Y_test.reshape(-1, 1, 1)

np.expand_dims(X_train, axis=0)
np.expand_dims(Y_train, axis=0)
np.expand_dims(X_test, axis=0)
np.expand_dims(Y_test, axis=0)

train_data = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
valid_data = tf.data.Dataset.from_tensor_slices((X_test, Y_test))

np.expand_dims(train_data, axis=0)
np.expand_dims(valid_data, axis=0)

model.add(LSTM(259, activation='tanh', dropout = 0.1, recurrent_dropout = 0.2, input_shape=(1, 259), return_sequences=True))
model.add(LSTM(520, activation='tanh', dropout = 0.1, recurrent_dropout = 0.2, return_sequences=True))
model.add(LSTM(256, dropout = 0.1, recurrent_dropout = 0.2, return_sequences=True))
model.add(Flatten())
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.4))
model.add(Dense(1, activation='softmax'))

#model.add(LSTM(128, activation='relu', input_shape=(1, 260), return_sequences=True))
#model.add(LSTM(64, activation='relu'))
#model.add(Dense(32, activation='relu'))
#model.add(Dense(1, activation='sigmoid'))

#model.add(LSTM(128, activation='relu', input_shape=(1, 260), return_sequences=True))
#model.add(Dropout(0.2))
#model.add(LSTM(64, activation='relu'))
#model.add(Dropout(0.2))
#model.add(Dense(32, activation='relu'))
#model.add(Dropout(0.2))
#model.add(Dense(1, activation='sigmoid'))

opt = tf.keras.optimizers.Adamax(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

#print(X_train.shape)
#print(Y_train.shape)

#model.fit(X_train, Y_train, epochs=10, validation_data= valid_data)

model.fit(X_train, Y_train, epochs=1000, validation_data=(X_test, Y_test))

Epoch 1/1000
32/32 [==============================] - 12s 59ms/step - loss: -150.4161 - accuracy: 0.0699 - val_loss: -554.1083 - val_accuracy: 0.0671
Epoch 2/1000
32/32 [==============================] - 1s 35ms/step - loss: -750.2553 - accuracy: 0.0638 - val_loss: -1163.1870 - val_accuracy: 0.0671
Epoch 3/1000
32/32 [==============================] - 1s 35ms/step - loss: -1420.7037 - accuracy: 0.0652 - val_loss: -1945.5033 - val_accuracy: 0.0671
Epoch 4/1000
32/32 [==============================] - 1s 35ms/step - loss: -2317.8343 - accuracy: 0.0566 - val_loss: -2892.0779 - val_accuracy: 0.0671
Epoch 5/1000
32/32 [==============================] - 1s 34ms/step - loss: -3384.2336 - accuracy: 0.0649 - val_loss: -4016.1174 - val_accuracy: 0.0671
Epoch 6/1000
32/32 [==============================] - 1s 33ms/step - loss: -4597.1173 - accuracy: 0.0649 - val_loss: -5294.7212 - val_accuracy: 0.0671
Epoch 7/1000
32/32 [==============================] - 1s 33ms/step - loss: -5859.2937 - accuracy

In [ ]:
from sklearn.metrics import classification_report

Y_pred = model.predict(X_test)
print(classification_report(Y_test, Y_pred))

ValueError: ignored

In [ ]:
# make predictions
trainPredict = model.predict(X_train)
testPredict = model.predict(X_test)
# calculate root mean squared error
trainScore = np.sqrt(mean_squared_error(Y_train[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = np.sqrt(mean_squared_error(Y_test[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

ValueError: ignored

Visualization of the Model 

In [ ]:
%matplotlib inline
cm = confusion_matrix(y_true=Y_test, y_pred=rounded_predictions)
plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues)

NameError: ignored

Saving the model 

In [ ]:
from keras.models import model_from_json

# serialize model to JSON
model_json = model.to_json()
with open("LSTM_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("LSTM_model.h5")
print("Saved model to disk")